In [20]:
import json
import boto3
import logging
import asyncio
from aiohttp import ClientSession
import urllib.request
import numpy as np
from pprint import pprint
from ept_info import Info

form = logging.Formatter("%(asctime)s : %(levelname)-5.5s : %(message)s")
logger = logging.getLogger()


consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(form)
logger.addHandler(consoleHandler)

logger.setLevel(logging.INFO)

s3 = boto3.client("s3")
bucket = "usgs-lidar-public"
bucket_url = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/"

def list_folders(s3_client: str, bucket_name: str):
    logger.info(f"fetching folders in {bucket_name}")
    paginator = s3_client.get_paginator('list_objects_v2')
    response_iterator = paginator.paginate(Bucket=bucket_name, Delimiter='/',
                                           Prefix='')

    for page in response_iterator:
        for content in page.get("CommonPrefixes", []):
            yield content.get('Prefix')

async def fetch(region, url, session):
    async with session.get(url) as response:
        return (region, await response.read())

async def run():
    regions = list_folders(s3, bucket)
    region_info = []
    async with ClientSession() as session:
        logger.info("loading the ept.json files")
        for region in regions:
            logger.info(f"reading {region[:-1]}'s ept.json file")
            if region == "USGS_LPC_WA_Western_North_2016_LAS_2018/" or \
                    region == "USGS_LPC_WA_Western_South_2016_LAS_2018/":
                ept_json_path = bucket_url + region + "ept-1.json"
            else:
                ept_json_path = bucket_url + region + "ept.json"
            ept_region_info = asyncio.ensure_future(fetch(region, ept_json_path, session))
            region_info.append(ept_region_info)
            logger.info(f"loaded {region[:-1]}'s ept.json file")

        response = await asyncio.gather(*region_info)

    return region_info

def load_ept_json() -> dict:
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(run())
    regions = loop.run_until_complete(future)

    region_ept_info = {}

    for i in range(len(regions)):
        try:
            region_ept_info[regions[i].result()[0]] = Info(regions[i].result()[1].decode())
        except json.decoder.JSONDecodeError as e:
            print(regions[i].result()[0])

    pprint(region_ept_info)

In [19]:
region_info

NameError: name 'region_ept_info' is not defined

In [15]:
import boto3
s3_client = boto3.client('s3')
response = s3.list_objects(Bucket="usgs-lidar-public", Delimiter='/', MaxKeys=2000)
request_files = response["CommonPrefixes"]
print(response)
# response = s3_client.get_object(Bucket='usgs-lidar-public', Key='ept.json')

{'ResponseMetadata': {'RequestId': 'ARZ63H36190SBQSV', 'HostId': 'LLTr02LdhpSbju5YptwKbbDqiRZocy3bECPGA0TZA0SA8jegslGL/TImOqROA0G3ebHh7f1CKe0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'LLTr02LdhpSbju5YptwKbbDqiRZocy3bECPGA0TZA0SA8jegslGL/TImOqROA0G3ebHh7f1CKe0=', 'x-amz-request-id': 'ARZ63H36190SBQSV', 'date': 'Tue, 17 Aug 2021 21:08:15 GMT', 'x-amz-bucket-region': 'us-west-2', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': True, 'Marker': '', 'NextMarker': 'USGS_LPC_LA_SoTerrebonne_GI_2015_LAS_2016/', 'Name': 'usgs-lidar-public', 'Prefix': '', 'Delimiter': '/', 'MaxKeys': 2000, 'CommonPrefixes': [{'Prefix': 'AK_BrooksCamp_2012/'}, {'Prefix': 'AK_Coastal_2009/'}, {'Prefix': 'AK_Fairbanks-NSBorough_2010/'}, {'Prefix': 'AK_Juneau_2012/'}, {'Prefix': 'AK_Kenai_2008/'}, {'Prefix': 'AK_MatanuskaSusitna-Lot1_2011/'}, {'Prefix': 'AK_MatanuskaSusitna-Lot2_2011/'}, {'Prefix': 'AK_NomeCreek_2010/'}, {'Pr